In [1]:
import numpy as np
from numba import cuda

# Define the CUDA kernel for matrix multiplication
@cuda.jit
def matrix_multiply_kernel(A, B, result):
    row, col = cuda.grid(2)
    
    if row < result.shape[0] and col < result.shape[1]:
        tmp = 0.0
        for i in range(A.shape[1]):
            tmp += A[row, i] * B[i, col]
        result[row, col] = tmp

# nhan ma tran tren gpu
def matrix_multiply_cuda(A, B):
    m, n = A.shape
    n, k = B.shape
    
    # cap bo nho cho gpu
    d_result = cuda.device_array((m, k), dtype=np.float32)

    # tao so block va grid su dung 
    blockdim = (32, 32)
    griddim = (int(np.ceil(m / blockdim[0])), int(np.ceil(k / blockdim[1])))

    # truyen du lieu ma tran tu host xuong device de xu ly 
    d_A = cuda.to_device(A)
    d_B = cuda.to_device(B)

    # goi lai ham voi so grid va block chia san de tinh
    matrix_multiply_kernel[griddim, blockdim](d_A, d_B, d_result)

    # tra ket qua tinh toan tu device ve cho  
    result_cuda = d_result.copy_to_host()

    return result_cuda

A = np.random.rand(1024, 1024).astype(np.float32)
B = np.random.rand(1024, 1024).astype(np.float32)

result = matrix_multiply_cuda(A, B)
print(result)
print(result.shape)

[[260.94797 252.42181 264.20285 ... 265.48074 256.1321  260.28616]
 [248.83034 242.30717 246.4565  ... 249.4068  249.12015 245.11578]
 [260.74115 254.29033 264.377   ... 262.57205 266.65427 260.8511 ]
 ...
 [263.20197 255.4851  255.12476 ... 262.40002 259.0676  253.65948]
 [260.50217 253.49136 255.64014 ... 258.29272 266.07153 253.83916]
 [255.8441  246.54317 255.18039 ... 257.9922  259.5996  254.05682]]
(1024, 1024)
